# HayStack Question Generation

* Para1: 7 Questions:
* Para2: 11 Questions: 22 s ± 953 ms
* Para3: 13 Questions: 25.5 s ± 51.3 ms
* Para4: 7 Questions: 15.3 s ± 27.7 ms
* Para5: 20 Questions: 56 s ± 2.68 s

On an average around 25s per paragraph.
Question Quality Similar to t5 model.
Number of questions not in control. Depends on the size of the paragraph.

In [ ]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-thpiie_4/farm-haystack_058486e437f04e08be42374e2782ddc6
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-thpiie_4/farm-haystack_058486e437f04e08be42374e2782ddc6
  Resolved https://github.com/deepset-ai/haystack.git to commit a0d7817dd59e310b90953c83fb25f63ceaa5ea99
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
# Imports needed to run this notebook

from pprint import pprint
from tqdm.auto import tqdm
from haystack.nodes import QuestionGenerator, BM25Retriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.utils import launch_es, print_questions

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
contexts = [
    "The competition formed as the FA Premier League on 20 February 1992 following the decision of clubs in the Football League First Division to break away from the Football League, which was originally founded in 1888, and take advantage of a lucrative television rights deal. The deal was worth £1 billion a year domestically as of 2013–14, with BSkyB and BT Group securing the domestic rights to broadcast 116 and 38 games respectively. The league generates €2.2 billion per year in domestic and international television rights. In 2014/15, teams were apportioned revenues of £1.6 billion.",
    "Puberty occurs through a long process and begins with a surge in hormone production, which in turn causes a number of physical changes. It is the stage of life characterized by the appearance and development of secondary sex characteristics (for example, a deeper voice and larger adam's apple in boys, and development of breasts and more curved and prominent hips in girls) and a strong shift in hormonal balance towards an adult state. This is triggered by the pituitary gland, which secretes a surge of hormonal agents into the blood stream, initiating a chain reaction to occur. The male and female gonads are subsequently activated, which puts them into a state of rapid growth and development; the triggered gonads now commence the mass production of the necessary chemicals. The testes primarily release testosterone, and the ovaries predominantly dispense estrogen. The production of these hormones increases gradually until sexual maturation is met. Some boys may develop gynecomastia due to an imbalance of sex hormones, tissue responsiveness or obesity.",
    "Frédéric François Chopin (/\ˈʃoʊpæn/; French pronunciation: [fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March 1810 – 17 October 1849), born Fryderyk Franciszek Chopin,[n 1] was a Polish and French (by citizenship and birth of father) composer and a virtuoso pianist of the Romantic era, who wrote primarily for the solo piano. He gained and has maintained renown worldwide as one of the leading musicians of his era, whose \"poetic genius was based on a professional technique that was without equal in his generation.\" Chopin was born in what was then the Duchy of Warsaw, and grew up in Warsaw, which after 1815 became part of Congress Poland. A child prodigy, he completed his musical education and composed his earlier works in Warsaw before leaving Poland at the age of 20, less than a month before the outbreak of the November 1830 Uprising.",
    "In the Pre-Modern era, many people\'s sense of self and purpose was often expressed via a faith in some form of deity, be that in a single God or in many gods. Pre-modern cultures have not been thought of creating a sense of distinct individuality, though. Religious officials, who often held positions of power, were the spiritual intermediaries to the common person. It was only through these intermediaries that the general masses had access to the divine. Tradition was sacred to ancient cultures and was unchanging and the social order of ceremony and morals in a culture could be strictly enforced.",
    "Beyoncé\'s first solo recording was a feature on Jay Z\'s \"\'03 Bonnie & Clyde\" that was released in October 2002, peaking at number four on the U.S. Billboard Hot 100 chart. Her first solo album Dangerously in Love was released on June 24, 2003, after Michelle Williams and Kelly Rowland had released their solo efforts. The album sold 317,000 copies in its first week, debuted atop the Billboard 200, and has since sold 11 million copies worldwide. The album\'s lead single, \"Crazy in Love\", featuring Jay Z, became Beyoncé\'s first number-one single as a solo artist in the US. The single \"Baby Boy\" also reached number one, and singles, \"Me, Myself and I\" and \"Naughty Girl\", both reached the top-five. The album earned Beyoncé a then record-tying five awards at the 46th Annual Grammy Awards; Best Contemporary R&B Album, Best Female R&B Vocal Performance for \"Dangerously in Love 2\", Best R&B Song and Best Rap/Sung Collaboration for \"Crazy in Love\", and Best R&B Performance by a Duo or Group with Vocals for \"The Closer I Get to You\" with Luther Vandross.",
]

docs = []
for para in contexts:
  docs.append({"content": para})

# Initialize document store and write in the documents
document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)

# Initialize Question Generator
question_generator = QuestionGenerator()

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
ERROR:posthog:error uploading: [PostHog] <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
</body>
</html>
 (503)
Using sep_token, but it is not set yet.


In [ ]:
reader = FARMReader("deepset/roberta-base-squad2")
qag_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
ERROR:posthog:error uploading: [PostHog] <html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body>
<center><h1>503 Service Temporarily Unavailable</h1></center>
</body>
</html>
 (503)
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
for idx, document in enumerate(tqdm(document_store)):
    print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
    result = qag_pipeline.run(documents=[document])
    print_questions(result)

0it [00:00, ?it/s]

INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.
INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.



 * Generating questions and answers for document 0: The competition formed as the FA Premier League on 20 February 1992 following the decision of clubs ...



Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.


...

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

INFO:haystack.schema:Setting the ID manually. This might cause a mismatch with the ID that would be generated from the document content and id_hash_keys value.



Generated pairs:
 - Q: What did Pre-Modern people express their sense of self and purpose via?
      A: a faith in some form of deity
 - Q: Pre-modern cultures have not been thought of creating a sense of what?
      A: distinct individuality
 - Q: Religious officials are often thought of as what type of person?
      A: spiritual intermediaries to the common person
 - Q: Who were spiritual intermediaries to the common person?
      A: Religious officials
 - Q: What was sacred to ancient cultures and was unchanging?
      A: Tradition
 - Q: What could be strictly enforced in ancient cultures?
      A: social order of ceremony and morals
 - Q: What was the social order of ceremony and morals in a culture?
      A: strictly enforced

 * Generating questions and answers for document 4: Beyoncé's first solo recording was a feature on Jay Z's "'03 Bonnie & Clyde" that was released in Oc...



Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]


Generated pairs:
 - Q: Beyonce's first solo album was a feature on what Jay Z song?
      A: Baby Boy
      A: "'03 Bonnie & Clyde
 - Q: When was Beyoncé's debut solo album released?
      A: June 24, 2003
      A: 46th Annual Grammy Awards
 - Q: What was the number four single on the US Billboard Hot 100 chart?
      A: Crazy in Love
      A: Baby Boy
 - Q: Who released her first album Dangerously in Love?
      A: Jay Z
      A: Beyoncé
 - Q: On what date did Beyonce release Crazy in Love?
      A: June 24, 2003
      A: 46th Annual Grammy
 - Q: How many copies did Crazy In Love sell in its first week?
      A: 317,000
      A: Beyoncé a then record-tying five
 - Q: What was the name of the lead single on Crazy in love?
      A: Baby Boy
 - Q: Who was the lead singer of Crazy in LOVE?
      A: Jay Z
      A: Luther Vandross
 - Q: On what day did the album come out?
      A: June 24, 2003
      A: Grammy Awards
 - Q: What was Beyonce's first number one single as a solo artist in the 